In [2]:
# import pandas as pd
# import numpy as np
# from statsmodels.tsa.arima.model import ARIMA
# from sklearn.preprocessing import MinMaxScaler
# from keras.models import Sequential
# from keras.layers import LSTM, Dense, Dropout
# # Load and preprocess data
# data = pd.read_csv('./Data/Yaerwise EX-IM trade.xlsx', index_col='date', parse_dates=['date'])
# scaler = MinMaxScaler()
# scaled_data = scaler.fit_transform(data)
# # Split data
# train_size = int(len(scaled_data) * 0.8)
# train, test = scaled_data[:train_size], scaled_data[train_size:]
# # ARIMA model
# arima_model = ARIMA(train, order=(1, 1, 1)).fit()
# arima_residuals = train - arima_model.fittedvalues
# # LSTM model
# def create_sequences(data, look_back=1):
#    X, y = [], []
#    for i in range(len(data) - look_back):
#        X.append(data[i:i + look_back])
#        y.append(data[i + look_back])
#    return np.array(X), np.array(y)
# look_back = 10
# X_train, y_train = create_sequences(arima_residuals, look_back)
# X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
# lstm_model = Sequential([
#    LSTM(50, return_sequences=True, input_shape=(look_back, 1)),
#    Dropout(0.2),
#    LSTM(50),
#    Dense(1)
# ])
# lstm_model.compile(optimizer='adam', loss='mse')
# lstm_model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=0)
# # Hybrid predictions
# arima_forecast = arima_model.forecast(steps=len(test))
# lstm_forecast = lstm_model.predict(X_train[-len(test):])
# hybrid_forecast = arima_forecast + lstm_forecast.flatten()
# # Inverse transform and evaluate
# hybrid_forecast = scaler.inverse_transform(hybrid_forecast.reshape(-1, 1))

In [3]:
import pandas as pd
import numpy as np


### Data processing

In [4]:
df = pd.read_excel('./Data/Yaerwise EX-IM trade.xlsx')
processed_data  = pd.DataFrame()
df.head()

,Unnamed: 0,Unnamed: 1,Year(2017-2018),Unnamed: 3,2018-19,Unnamed: 5,2019-2020,Unnamed: 7,2020-21,Unnamed: 9,2021-22,Unnamed: 11,2022-23,Unnamed: 13,2023-24,Unnamed: 15,2024-25,Unnamed: 17
0,NaN,Critical Mineral,Export($ million),Imort,Export($ million),Import,Export,Import,Export,Import,Export,Import,Export,Import,Export,Import,Export,Import
1,NaN,Antimony,15.06,11.44,17.87,9.67,13.08,9.37,12.63,8.01,20.26,13.38,19.32,19.19,15.94,19.74,40.9,38.16
2,NaN,"Beryllium,hafnium, indium,gallium,rhenium",0.58,24.65,0.9,35.75,1.87,19.62,1.62,17.58,3.28,29.12,2.26,71.43,3.05,76.31,5.52,76.93
3,NaN,Bismuth,0.07,3.19,0.01,2.77,0.01,1.53,0.03,1.83,0.61,2.54,0.12,3.34,0.02,3.19,0.03,5.55
4,NaN,Cobalt,3.93,58.99,7.9,75.8,3.6,67.25,6.54,44.77,13.82,88.49,28.29,110.5,22.84,113.18,18.44,96.1


In [5]:
imp = []
exp = []
for i in range(1,len(df.iloc[1:,1:])+1):
    exp.append((df.iloc[i,2::2].values).astype('float32'))
    imp.append((df.iloc[i,3::2].values).astype('float32'))

processed_data['Mineral'] = (df.iloc[1:,1].values).astype('str')
processed_data['Export'] = exp
processed_data['Import'] = imp

# len((df.iloc[1:,1].values).astype('str')),len(exp),len(imp)


In [6]:
processed_data.to_csv('./processed_data.csv',index=False)

### Model Preperation 

In [7]:
import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout

d:\hackthon\mineral forcasting\.env\Lib\site-packages\keras\src\export\tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


In [9]:
import warnings

warnings.filterwarnings('ignore')

def forcast_armima(data_,steps=2):
    data = data_['Export'].values
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(data[0].reshape(-1, 1))
    arima_model = ARIMA(scaled_data, order=(5, 1, 0)).fit()
    presiction = arima_model.forecast(steps=steps)
    original_prediction = scaler.inverse_transform(presiction.reshape(-1, 1))
    # print('next two year export',original_prediction)
    data = data_['Import'].values
    scaler_imp = MinMaxScaler()
    scaled_data_imp = scaler_imp.fit_transform(data[0].reshape(-1, 1))
    arima_model_imp = ARIMA(scaled_data_imp, order=(5, 1, 0)).fit()
    presiction_imp = arima_model_imp.forecast(steps=steps)
    original_prediction_imp = scaler_imp.inverse_transform(presiction_imp.reshape(-1, 1))
    # print('next two year Import',original_prediction_imp)
    return original_prediction.T[0],original_prediction_imp.T[0]
forcast_armima(processed_data[processed_data["Mineral"]=='Antimony'],steps=2)

(array([26.9389133 , 34.39338525]), array([27.44611233, 54.0620579 ]))

In [25]:
import warnings
from statsmodels.tsa.arima.model import ARIMA
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.metrics import mean_squared_error, r2_score
import warnings

warnings.filterwarnings('ignore')

def train_arima(series, order=(5,1,0)):
    model = ARIMA(series, order=order)
    model_fit = model.fit()
    return model_fit
def arima_residuals(series, arima_model):
    fitted = arima_model.fittedvalues
    residuals = series[len(series)-len(fitted):] - fitted
    return residuals

def create_lstm_data(data, lookback=5):
    X, y = [], []
    for i in range(len(data)-lookback):
        X.append(data[i:i+lookback])
        y.append(data[i+lookback])
    return np.array(X), np.array(y)

def build_lstm(input_shape):
    model = Sequential([
        LSTM(50, activation='tanh', input_shape=input_shape),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse')
    return model

def hybrid_arima_lstm(series, steps=2, lookback=5):
    warnings.filterwarnings('ignore')
    series = np.asarray(series).astype(float)

    # ---- ARIMA ----
    arima_model = train_arima(series)
    arima_forecast = arima_model.forecast(steps=steps)

    # ---- Residuals ----
    residuals = arima_residuals(series, arima_model)

    # ---- Scale residuals ----
    scaler = MinMaxScaler()
    residuals_scaled = scaler.fit_transform(residuals.reshape(-1,1))

    # ---- LSTM data ----
    X, y = create_lstm_data(residuals_scaled, lookback)
    X = X.reshape((X.shape[0], X.shape[1], 1))

    # ---- Train LSTM ----
    lstm = build_lstm((lookback,1))
    lstm.fit(X, y, epochs=50, batch_size=8, verbose=0)

    # ---- Predict residuals ----
    last_seq = residuals_scaled[-lookback:].reshape((1, lookback, 1))
    lstm_residual_pred = []

    for _ in range(steps):
        pred = lstm.predict(last_seq, verbose=0)
        lstm_residual_pred.append(pred[0,0])
        last_seq = np.append(last_seq[:,1:,:], pred.reshape(1,1,1), axis=1)

    lstm_residual_pred = scaler.inverse_transform(
        np.array(lstm_residual_pred).reshape(-1,1)
    ).flatten()

    # ---- Final Hybrid Forecast ----
    hybrid_forecast = arima_forecast + lstm_residual_pred

    return hybrid_forecast,arima_forecast

def hybrid(series,step = 2):
    export_series = series['Export'].values
    
    import_series = series['Import'].values

# Example: forecast for the first mineral (index 0)
    print(export_series[0])
    export_pred = hybrid_arima_lstm(export_series[0], steps=2)
    import_pred = hybrid_arima_lstm(import_series[0], steps=2)
# print("Next 2-year Export Forecast:", export_pred)
# print("Next 2-year Import Forecast:", import_pred)
    return export_pred,import_pred
hybrid(processed_data[processed_data["Mineral"]=='Antimony'],step=2)

[15.06 17.87 13.08 12.63 20.26 19.32 15.94 40.9 ]


((array([34.21352447, 43.67153761]), array([26.93833459, 34.39284632])),
 (array([31.81892623, 60.10404897]), array([27.62916793, 54.07375503])))

In [ ]:
# series = processed_data[processed_data["Mineral"]=='Antimony']
# export_series = series['Export'].values  
# import_series = series['Import'].values
# export_pred = hybrid_arima_lstm(export_series[0][:7], steps=2)
# import_pred = hybrid_arima_lstm(import_series[0][:7], steps=2)
# export_true = export_series[0][7:]
# import_true = import_series[0][7:]
# export_pred,import_pred,export_true,import_true


((array([19.07313861, 17.21518461]), array([19.67462332, 17.70042808])),
 (array([21.97193149, 21.65159077]), array([22.10047973, 21.86090531])),
 array([40.9], dtype=float32),
 array([38.16], dtype=float32))